## 자바스크립트에서 비동기 처리

### 비동기 - 자바스크립트는 싱글 스레드로 동작하기에 작업은 차례로 실행되며, 이전 작업이 완료될 때까지 대기
### &nbsp;&nbsp; 하지만 비동기 프로그래밍에는 임의의 순서로 또는 동시에 작업 진행가능

- 콜백: 함수의 파라미터로 함수를 전달하며, 비동기 처리가 끝났을 때 전달된 함수 실행
- 프로미스: 비동기 작업이 완료되면 결과를 반환, 성공과 실패에 대한 처리 진행
- 어싱크 어웨이트: 프로미스를 사용하는 비동기 작업을 동기적으로 처리하는 것처럼 코드 작성

### 1) 콜백 함수
#### 가독성이 좋지 않으며 유지보수 및 디버깅이 어려움

In [1]:
// 콜백 예제
const DB=[];

// 회원가입 API 함수
function register(user){    // 콜백이 3중으로 중첩된 함수
    return saveDB(user, function(user){         // 콜백 1 
        return sendEmail(user, function(user){  // 콜백 2
            return getResult(user);             // 콜백 3
        });
    });
}

// DB에 저장 후 콜백 실행
function saveDB(user, callback){
    DB.push(user);
    console.log(`save ${user.name} to DB`);
    return callback(user);
}

// 이메일 발송 로그만 남기는 코드 실행 후 콜백 실행
function sendEmail(user, callback){
    console.log(`email to ${user.email}`);
    return callback(user);
}

// 결과를 반환하는 함수
function getResult(user){
    return `success register ${user.name}`;
}

const result=register({email: "andy@test.com", password: "1234", name: "andy"});
console.log(result);    
/*
save andy to DB
email to andy@test.com
success register andy
*/



save andy to DB
email to andy@test.com
success register andy


### 2) Promise 객체
#### 비동기 실행을 동기화하는 구문으로 사용
#### 이행, 대기, 거절 세 가지의 상태를 가짐

In [ ]:
// Promise 예제

const DB=[];

function saveDB(user){
    const oldDBSize=DB.length;
    DB.push(user);
    console.log(`save ${user.name} to DB`);
    return new Promise((resolve, reject) => {   // 콜백 대신 Promise 객체 반환
        if(DB.length>oldDBSize){
            resolve(user);  // 성공 시 유저 정보 반환
        }else{
            reject(new Error("Save DB Error!")); // 실패 시 에러 발생
        }
    });
}

function sendEmail(user){
    console.log(`email to ${user.email}`);
    return new Promise((resolve) => {   // Promise 객체를 반환. 실패 처리 없음
        resolve(user); 
    });
}

function getResult(user){
    return new Promise((resolve, reject) => {   // Promise 객체 반환
        resolve(`success register ${user.name}`)    // 성공 시 성공 메시지와 유저명 반환
    })
}

function registerByPromise(user){
    // 비동기 호출이지만 , 순서를 지켜서 실행
    const result=saveDB(user).then(sendEmail).then(getResult);
    // 아직 완료되지 않았으므로 지연(pending) 상태
    console.log(result);
    return result;
}

const myUser={email: "andy@test.com", password: "1234", name: "andy"};
const result=registerByPromise(myUser);
// 결과값이 Promise이므로 then() 메서드에 함수를 넣어서 결과값을 볼 수 있음(Promise 객체에만 then 사용가능)
result.then(console.log);
/* save andy to DB
Promise { <pending> }
Promise { <pending> }
email to andy@test.com
success register andy */

save andy to DB
Promise { <pending> }


Promise { <pending> }

email to andy@test.com
success register andy


#### 동시에 여러 Promise 객체 호출

In [ ]:
// 동시에 여러 Promise 객체 호출하는 코드
const DB=[];

function saveDB(user){
    const oldDBSize=DB.length;
    DB.push(user);
    console.log(`save ${user.name} to DB`);
    return new Promise((resolve, reject) => {   // 콜백 대신 Promise 객체 반환
        if(DB.length>oldDBSize){
            resolve(user);  // 성공 시 유저 정보 반환
        }else{
            reject(new Error("Save DB Error!")); // 실패 시 에러 발생
        }
    });
}

function sendEmail(user){
    console.log(`email to ${user.email}`);
    return new Promise((resolve) => {   // Promise 객체를 반환. 실패 처리 없음
        resolve(user); 
    });
}

function getResult(user){
    return new Promise((resolve, reject) => {   // Promise 객체 반환
        resolve(`success register ${user.name}`)    // 성공 시 성공 메시지와 유저명 반환
    })
}

function registerByPromise(user){
    // 비동기 호출이지만 , 순서를 지켜서 실행
    const result=saveDB(user).then(sendEmail).then(getResult);
    // 아직 완료되지 않았으므로 지연(pending) 상태
    console.log(result);
    return result;
}

// const myUser={email: "andy@test.com", password: "1234", name: "andy"};
// const result=registerByPromise(myUser);
// // 결과값이 Promise이므로 then() 메서드에 함수를 넣어서 결과값을 볼 수 있음(Promise 객체에만 then 사용가능)
// result.then(console.log);

// 수정 코드
const myUser={email: "andy@test.com", password: "1234", name: "andy"};
allResult=Promise.all([saveDB(myUser), sendEmail(myUser), getResult(myUser)]);
allResult.then(console.log)

/*
save andy to DB
email to andy@test.com
Promise { <pending> }
[
  { email: 'andy@test.com', password: '1234', name: 'andy' },
  { email: 'andy@test.com', password: '1234', name: 'andy' },
  'success register andy'
]
  */

save andy to DB
email to andy@test.com


Promise { <pending> }

[
  { email: 'andy@test.com', password: '1234', name: 'andy' },
  { email: 'andy@test.com', password: '1234', name: 'andy' },
  'success register andy'
]


#### Promise 예외 처리

In [ ]:
// Promise 예외처리 코드
const DB=[];

function saveDB(user){
    // const oldDBSize=DB.length;    
    const oldDBSize=DB.length+1;    // 오류가 발생하도록 수정
    DB.push(user);
    console.log(`save ${user.name} to DB`);
    return new Promise((resolve, reject) => {   // 콜백 대신 Promise 객체 반환
        if(DB.length>oldDBSize){
            resolve(user);  // 성공 시 유저 정보 반환
        }else{
            reject(new Error("Save DB Error!")); // 실패 시 에러 발생
        }
    });
}

function sendEmail(user){
    console.log(`email to ${user.email}`);
    return new Promise((resolve) => {   // Promise 객체를 반환. 실패 처리 없음
        resolve(user); 
    });
}

function getResult(user){
    return new Promise((resolve, reject) => {   // Promise 객체 반환
        resolve(`success register ${user.name}`)    // 성공 시 성공 메시지와 유저명 반환
    })
}

function registerByPromise(user){
    // 비동기 호출이지만 , 순서를 지켜서 실행
    // const result=saveDB(user).then(sendEmail).then(getResult);

    const result=saveDB(user)
                    .then(sendEmail)
                    .then(getResult)
                    .catch(error => new Error(error));  // 예외처리구문 추가
    // 아직 완료되지 않았으므로 지연(pending) 상태
    console.log(result);
    return result;
}

// const myUser={email: "andy@test.com", password: "1234", name: "andy"};
// const result=registerByPromise(myUser);
// // 결과값이 Promise이므로 then() 메서드에 함수를 넣어서 결과값을 볼 수 있음(Promise 객체에만 then 사용가능)
// result.then(console.log);

// 수정 코드
const myUser={email: "andy@test.com", password: "1234", name: "andy"};
allResult=Promise.all([saveDB(myUser), sendEmail(myUser), getResult(myUser)]);
allResult.then(console.log)

/*
save andy to DB
email to andy@test.com
Promise { <pending> }
[
  { email: 'andy@test.com', password: '1234', name: 'andy' },
  { email: 'andy@test.com', password: '1234', name: 'andy' },
  'success register andy'
]
  */

save andy to DB
email to andy@test.com


Promise { <pending> }

Error: Save DB Error!
    at evalmachine.<anonymous>:13:20
    at new Promise (<anonymous>)
    at saveDB (evalmachine.<anonymous>:9:12)
    at evalmachine.<anonymous>:51:24
    at Script.runInThisContext (node:vm:122:12)
    at Object.runInThisContext (node:vm:296:38)
    at run ([eval]:1020:15)
    at onRunRequest ([eval]:864:18)
    at onMessage ([eval]:828:13)
    at process.emit (node:events:514:28)

In [1]:
// finally 추가

// Promise 예외처리 코드
const DB=[];

function saveDB(user){
    // const oldDBSize=DB.length;    
    const oldDBSize=DB.length+1;    // 오류가 발생하도록 수정
    DB.push(user);
    console.log(`save ${user.name} to DB`);
    return new Promise((resolve, reject) => {   // 콜백 대신 Promise 객체 반환
        if(DB.length>oldDBSize){
            resolve(user);  // 성공 시 유저 정보 반환
        }else{
            reject(new Error("Save DB Error!")); // 실패 시 에러 발생
        }
    });
}

function sendEmail(user){
    console.log(`email to ${user.email}`);
    return new Promise((resolve) => {   // Promise 객체를 반환. 실패 처리 없음
        resolve(user); 
    });
}

function getResult(user){
    return new Promise((resolve, reject) => {   // Promise 객체 반환
        resolve(`success register ${user.name}`)    // 성공 시 성공 메시지와 유저명 반환
    })
}

function registerByPromise(user){
    // 비동기 호출이지만 , 순서를 지켜서 실행
    // const result=saveDB(user).then(sendEmail).then(getResult);

    const result=saveDB(user)
                    .then(sendEmail)
                    .then(getResult)
                    .catch(error => new Error(error))  // 예외처리구문 추가
                    // 성공, 실패 여부에 관계없이 실행
                    .finally(() => console.log("완료!"));
    // 아직 완료되지 않았으므로 지연(pending) 상태
    console.log(result);
    return result;
}

// const myUser={email: "andy@test.com", password: "1234", name: "andy"};
// const result=registerByPromise(myUser);
// // 결과값이 Promise이므로 then() 메서드에 함수를 넣어서 결과값을 볼 수 있음(Promise 객체에만 then 사용가능)
// result.then(console.log);

// 수정 코드
const myUser={email: "andy@test.com", password: "1234", name: "andy"};
allResult=Promise.all([saveDB(myUser), sendEmail(myUser), getResult(myUser)]);
allResult.then(console.log)

/*
save andy to DB
email to andy@test.com
Promise { <pending> }
[
  { email: 'andy@test.com', password: '1234', name: 'andy' },
  { email: 'andy@test.com', password: '1234', name: 'andy' },
  'success register andy'
]
  */

save andy to DB
email to andy@test.com


Promise { <pending> }

Error: Save DB Error!
    at evalmachine.<anonymous>:15:20
    at new Promise (<anonymous>)
    at saveDB (evalmachine.<anonymous>:11:12)
    at evalmachine.<anonymous>:55:24
    at Script.runInThisContext (node:vm:122:12)
    at Object.runInThisContext (node:vm:296:38)
    at run ([eval]:1020:15)
    at onRunRequest ([eval]:864:18)
    at onMessage ([eval]:828:13)
    at process.emit (node:events:514:28)

#### 프로미스의 문제점

In [ ]:
// 이상적인 Promise 코드
function goodPromise(val){
    // Promise를 생성 후 반환
    return new Promise((resolve, reject) => {
        resolve(val);
    });
}

goodPromise("세상에")
    // Promise에서 resolve 이후에는 then 호출가능
    .then((val) => {
        return val+" 이런";
    })
    .then((val) => {
        return val+" 코드는";
    })
    .then((val) => {
        return val+" 없습니다";
    })
    .then((val) => {
        console.log(val);
    })
    .catch((err) => {   // Promise에서 reject가 호출되었을 경우 실행
        console.log(err);
    });

Promise { <pending> }

세상에 이런 코드는 없습니다


In [ ]:
// 복잡한 Promise 예제
// 현재 상영 영화 순위를 20위까지 Promise를 사용해 확인하기

const axios=require("axios");   // axios 임포트
// 영화 순위 정보 url
const url="https://raw.githubusercontent.com/wapj/jsbackend/main/movieinfo.json";

axios
    .get(url)   // GET 요청
    .then((result) => {     // 결과값 처리
        if(result.status!=200){
            throw new Error("요청에 실패했습니다.");
        }

        if(result.data){    // result.data가 있으면 결과를 반환
            return result.data;
        }

        throw new Error("데이터 없습니다."); // data가 없으면 에러
    })
    .then((data) => {   // data 처리
        if(!data.articleList || data.articleList.size==0){  // 크기가 0이면 에러
            throw new Error("데이터가 없습니다.");
        }
        return data.articleList;    // 영화 리스트 반환
    })
    .then((articles) => {
        return articles.map((article, idx) => { // 영화 리스트를 제목과 순위 정보로 분리
            return {title: article. title, rank: idx+1}
        });
    })
    .then((results) => {
        for(let movieInfo of results){  // 받은 영화 리스트 정보 출력
            console.log(`[${movieInfo.rank}위] ${movieInfo.title}`);
        }
    })
    .catch((err) => {   // 중간에 발생한 에러들을 여기서 처리
        console.log("<<에러발생>>");
        console.error(err);
    });

Promise { <pending> }

[1위] 처음부터 잘했으면 얼마나 좋니
[2위] <본즈 앤 올> 궁지로 내몰린 10대를 보는 시선
[3위] 경이로운 생生의 의지로 창조해낸 ‘페르시아어’
[4위] 뻔하지 않은 사랑 영화
[5위] 우린 아무것도 모른다, 틀렸다는것만 증명할 뿐…에올
[6위] [영화리뷰] <더 메뉴>를 보고
[7위] 진실한 삶의 태도를 제시하다
[8위] 중요한 건 꺾이지 않는 마음
[9위] 치즈버거 세트의 행복
[10위] 즐기거나 놀리거나,
[11위] [영화감상]오늘 밤, 세계에서 이 사랑이 사라진다 해도
[12위] 아들을 구하고 싶다면 달려라
[13위] 인생은 아름다워(2022)
[14위] 영화 <사도>, 죽음의 문턱에 와서야 닿는 마음에 대해
[15위] 스타워즈: 안도르
[16위] 죽음의 문턱에서 거짓말로 살아남은 자의 고백
[17위] 닫힌 마음 - 영화 '체리향기'
[18위] <오늘 밤, 세계에서 이 사랑이 사라진다 해도> 리뷰
[19위] <더 메뉴> 180만 원짜리 먹으러 와서 사레 걸린기분
[20위] 닭장을 나온 백호 


In [ ]:
// 문제점 및 대안 찾기
// 콜백보다는 깔끔한 코드를 유지할 수 있지만 문제점 발생
// ex 1) then() 함수에 성공 시와 실패 시 처리할 함수를 둘 다 넘기는 경우
function myWork(work){
    return new Promise((resolve, reject) => {
        if(work==='done'){
            resolve('게임가능');
        }else{
            reject(new Error("게임 불가능"));
        }
        
    })
}

// 콜백과 다를 바가 없음
myWork('done').then(function(value){console.log(value)}, function(err){console.error(err)});

// 에러처리해 개선
myWork('doing')
    .then(function (value) {console.log(value)})
    .catch(function(err) {console.error(err)});

In [3]:
// ex 2) Promise를 중첩해서 사용하는 경우
function myWork(work){
    return new Promise((resolve, reject) => {
        resolve(work.toUpperCase());
    })
}

function playGame(work){
    return new Promise((resolve, reject) => {
        if(work === 'DONE'){
            resolve("GO PLAY GAME");
        }else{
            reject(new Error("DON'T"));
        }
    })
}

// Promise를 중첩해서 사용
myWork('done')
    .then(function(result){
        playGame(result).then(function(val){
            console.log(val);
        });
    })

// 결과를 then으로 넘겨 개선
myWork('done')
    .then(playGame)
    .then(console.log)

Promise { <pending> }

GO PLAY GAME
GO PLAY GAME


### 3) async await 구문

In [ ]:
// async 예제
async function myName(){    // async: Promise로 반환값을 감싸서 넘겨줌
    return "Andy";
}

console.log(myName());

Promise { 'Andy' }


In [ ]:
// async await 예제
async function myName(){
    return "Andy";
}

async function showName(){  // 이름을 출력하는 함수
    const name=await myName();
    console.log(name);
}

console.log(showName());    // 콘솔에 이름 출력

Promise { <pending> }


Andy


In [ ]:
// async, await, setTimeout()으로 1부터 10까지 세기
function waitOneSecond(msg){    // 1초 대기하고 메시지 출력
    return new Promise((resolve, _) => {
        setTimeout(() => resolve(`${msg}`), 1000);
    });
}

async function countOneToTen(){ // 10초 동안 1초마다 메시지 출력
    for(let x of [...Array(10).keys()]){    // 0부터 9까지 루프를 순회
        // 1초 대기 후 result에 결과값 저장
        let result=await waitOneSecond(`${x+1}초 대기 중...`);
        console.log(result);
    }
    console.log("완료");
}

countOneToTen();

Promise { <pending> }

1초 대기 중...
2초 대기 중...
3초 대기 중...
4초 대기 중...
5초 대기 중...
6초 대기 중...
7초 대기 중...
8초 대기 중...
9초 대기 중...
10초 대기 중...
완료


In [ ]:
// Top 20 영화 제목 가져오기 async await로 리팩터링
const axios=require("axios");

async function getTop20Movies(){    // await를 사용하므로 async를 붙임
    const url="https://raw.githubusercontent.com/wapj/jsbackend/main/movieinfo.json";
    try{
        // 네트워크에서 데이터를 받아오므로 await로 기다림
        const result=await axios.get(url);
        const {data}=result;    // 결과값에는 data 프로퍼티가 있음
        // data 또는 articleList 없을 때 예외처리
        if(!data.articleList || data.articleList.size==0){
            throw new Error("데이터가 없습니다.");
        }
        // data에서 필요한 영화 제목과 순위 정보를 뽑아냄
        const movieInfos=data.articleList.map((article, idx) => {
            return {title: article.title, rank: idx+1};
        });

        // 데이터 출력
        for(let movieInfo of movieInfos){
            console.log(`[${movieInfo.rank}위] ${movieInfo.title}`);
        }
    } catch(err){
        // 예외 처리는 기존 코드와 같게 try catch로 감쌈
        throw new Error(err);
    }
}

// await를 함수 안에서만 사용가능하므로 함수를 하나 생성해 실행
getTop20Movies();

Promise { <pending> }

[1위] 처음부터 잘했으면 얼마나 좋니
[2위] <본즈 앤 올> 궁지로 내몰린 10대를 보는 시선
[3위] 경이로운 생生의 의지로 창조해낸 ‘페르시아어’
[4위] 뻔하지 않은 사랑 영화
[5위] 우린 아무것도 모른다, 틀렸다는것만 증명할 뿐…에올
[6위] [영화리뷰] <더 메뉴>를 보고
[7위] 진실한 삶의 태도를 제시하다
[8위] 중요한 건 꺾이지 않는 마음
[9위] 치즈버거 세트의 행복
[10위] 즐기거나 놀리거나,
[11위] [영화감상]오늘 밤, 세계에서 이 사랑이 사라진다 해도
[12위] 아들을 구하고 싶다면 달려라
[13위] 인생은 아름다워(2022)
[14위] 영화 <사도>, 죽음의 문턱에 와서야 닿는 마음에 대해
[15위] 스타워즈: 안도르
[16위] 죽음의 문턱에서 거짓말로 살아남은 자의 고백
[17위] 닫힌 마음 - 영화 '체리향기'
[18위] <오늘 밤, 세계에서 이 사랑이 사라진다 해도> 리뷰
[19위] <더 메뉴> 180만 원짜리 먹으러 와서 사레 걸린기분
[20위] 닭장을 나온 백호 
